<a href="https://colab.research.google.com/github/Omar-Eladawy/NLP/blob/main/M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace valid.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [6]:
!pip install
!pip install spacy
!pip install langdetect
!python -m spacy download en_core_web_sm

ERROR: You must give at least one requirement to install (see "pip help install")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import nltk
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
import csv
import pandas as pd
from collections import Counter
import math
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from spacy.tokens import Doc


In [9]:
s = pd.read_csv("train.csv",index_col=0,parse_dates=True)

In [10]:
s

,Title,Body,Score,ViewCount,Label,LabelNum
Id,,,,,,
197234,Drop\stop mobile data connection (non-wifi) by...,<p>Can I set Android 4.4.2 to drop mobile data...,0,34,android,0
114800,How to automatically crop text messages when S...,<p>Is there a way to prevent the Messages app ...,0,836,android,0
124532,Can't find text message that was to a group,<p>When John Doe texts to a group that include...,1,28,android,0
193875,Can't store contacts on my Android phone,<p>I was going through all of my installed app...,0,158,android,0
50332,Dropbox on Samsung Galaxy - where is the Setti...,"<p>On a Sony Xperia, the settings button in Dr...",1,963,android,0
...,...,...,...,...,...,...
101944,How to securely root g2 phone (AT&T LGE LG-D800)?,"<p>I've been looking to root my phone, only al...",4,4972,android,0
194088,Is Fennec F-Droid an official product by Mozilla?,"<p>Is the <a href=""https://f-droid.org/package...",2,3799,android,0
57764,Whats the difference between cell phone and da...,<p>Is cell phone and data/WiFi/LTE radiation t...,1,131,android,0


In [12]:
copy= s.copy()

In [13]:
copy.head()

,Title,Body,Score,ViewCount,Label,LabelNum
Id,,,,,,
197234,Drop\stop mobile data connection (non-wifi) by...,<p>Can I set Android 4.4.2 to drop mobile data...,0,34,android,0
114800,How to automatically crop text messages when S...,<p>Is there a way to prevent the Messages app ...,0,836,android,0
124532,Can't find text message that was to a group,<p>When John Doe texts to a group that include...,1,28,android,0
193875,Can't store contacts on my Android phone,<p>I was going through all of my installed app...,0,158,android,0
50332,Dropbox on Samsung Galaxy - where is the Setti...,"<p>On a Sony Xperia, the settings button in Dr...",1,963,android,0


In [14]:
copy.drop(columns=['Score'], inplace=True)


In [15]:
copy.drop(columns=['ViewCount'], inplace=True)

In [16]:
copy.reset_index(drop=True , inplace=True)

In [18]:
copy['merged_column'] = copy['Title'] + copy['Body']

In [19]:
copy

,Title,Body,Label,LabelNum,merged_column
0,Drop\stop mobile data connection (non-wifi) by...,<p>Can I set Android 4.4.2 to drop mobile data...,android,0,Drop\stop mobile data connection (non-wifi) by...
1,How to automatically crop text messages when S...,<p>Is there a way to prevent the Messages app ...,android,0,How to automatically crop text messages when S...
2,Can't find text message that was to a group,<p>When John Doe texts to a group that include...,android,0,Can't find text message that was to a group<p>...
3,Can't store contacts on my Android phone,<p>I was going through all of my installed app...,android,0,Can't store contacts on my Android phone<p>I w...
4,Dropbox on Samsung Galaxy - where is the Setti...,"<p>On a Sony Xperia, the settings button in Dr...",android,0,Dropbox on Samsung Galaxy - where is the Setti...
...,...,...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D800)?,"<p>I've been looking to root my phone, only al...",android,0,How to securely root g2 phone (AT&T LGE LG-D80...
51366,Is Fennec F-Droid an official product by Mozilla?,"<p>Is the <a href=""https://f-droid.org/package...",android,0,Is Fennec F-Droid an official product by Mozil...
51367,Whats the difference between cell phone and da...,<p>Is cell phone and data/WiFi/LTE radiation t...,android,0,Whats the difference between cell phone and da...
51368,“There are no android phones associated with t...,<p>I recently bought an eLocity A7 Internet Ta...,android,0,“There are no android phones associated with t...


In [20]:
copy.drop(columns=['Title'], inplace=True)
copy.drop(columns=['Body'], inplace=True)


In [21]:
copy

,Label,LabelNum,merged_column
0,android,0,Drop\stop mobile data connection (non-wifi) by...
1,android,0,How to automatically crop text messages when S...
2,android,0,Can't find text message that was to a group<p>...
3,android,0,Can't store contacts on my Android phone<p>I w...
4,android,0,Dropbox on Samsung Galaxy - where is the Setti...
...,...,...,...
51365,android,0,How to securely root g2 phone (AT&T LGE LG-D80...
51366,android,0,Is Fennec F-Droid an official product by Mozil...
51367,android,0,Whats the difference between cell phone and da...
51368,android,0,“There are no android phones associated with t...


In [22]:
copy=copy[['merged_column','Label','LabelNum']]
copy

,merged_column,Label,LabelNum
0,Drop\stop mobile data connection (non-wifi) by...,android,0
1,How to automatically crop text messages when S...,android,0
2,Can't find text message that was to a group<p>...,android,0
3,Can't store contacts on my Android phone<p>I w...,android,0
4,Dropbox on Samsung Galaxy - where is the Setti...,android,0
...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D80...,android,0
51366,Is Fennec F-Droid an official product by Mozil...,android,0
51367,Whats the difference between cell phone and da...,android,0
51368,“There are no android phones associated with t...,android,0


In [23]:
copy['merged_column'] = copy['merged_column'].map(lambda x: x.lstrip('</p>').rstrip('<p>'))

In [24]:
copy

,merged_column,Label,LabelNum
0,Drop\stop mobile data connection (non-wifi) by...,android,0
1,How to automatically crop text messages when S...,android,0
2,Can't find text message that was to a group<p>...,android,0
3,Can't store contacts on my Android phone<p>I w...,android,0
4,Dropbox on Samsung Galaxy - where is the Setti...,android,0
...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D80...,android,0
51366,Is Fennec F-Droid an official product by Mozil...,android,0
51367,Whats the difference between cell phone and da...,android,0
51368,“There are no android phones associated with t...,android,0


In [25]:
def remove_html_tags(text):
    clean_text = re.sub('<[^<]+?>', '', text)
    return clean_text

In [26]:
copy['merged_column'] = copy['merged_column'].apply(remove_html_tags)
print(copy.head())

                                       merged_column    Label  LabelNum
0  Drop\stop mobile data connection (non-wifi) by...  android         0
1  How to automatically crop text messages when S...  android         0
2  Can't find text message that was to a groupWhe...  android         0
3  Can't store contacts on my Android phoneI was ...  android         0
4  Dropbox on Samsung Galaxy - where is the Setti...  android         0


In [27]:
copy

,merged_column,Label,LabelNum
0,Drop\stop mobile data connection (non-wifi) by...,android,0
1,How to automatically crop text messages when S...,android,0
2,Can't find text message that was to a groupWhe...,android,0
3,Can't store contacts on my Android phoneI was ...,android,0
4,Dropbox on Samsung Galaxy - where is the Setti...,android,0
...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D80...,android,0
51366,Is Fennec F-Droid an official product by Mozil...,android,0
51367,Whats the difference between cell phone and da...,android,0
51368,“There are no android phones associated with t...,android,0


In [28]:
nlp = spacy.load('en_core_web_sm')


In [29]:
copy

,merged_column,Label,LabelNum
0,Drop\stop mobile data connection (non-wifi) by...,android,0
1,How to automatically crop text messages when S...,android,0
2,Can't find text message that was to a groupWhe...,android,0
3,Can't store contacts on my Android phoneI was ...,android,0
4,Dropbox on Samsung Galaxy - where is the Setti...,android,0
...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D80...,android,0
51366,Is Fennec F-Droid an official product by Mozil...,android,0
51367,Whats the difference between cell phone and da...,android,0
51368,“There are no android phones associated with t...,android,0


In [30]:
def tokenize_text(text):
  tokens = word_tokenize(text)
  return tokens
copy['merged_column_tokens'] = copy['merged_column'].apply(tokenize_text)

In [31]:
copy

,merged_column,Label,LabelNum,merged_column_tokens
0,Drop\stop mobile data connection (non-wifi) by...,android,0,"[Drop\stop, mobile, data, connection, (, non-w..."
1,How to automatically crop text messages when S...,android,0,"[How, to, automatically, crop, text, messages,..."
2,Can't find text message that was to a groupWhe...,android,0,"[Ca, n't, find, text, message, that, was, to, ..."
3,Can't store contacts on my Android phoneI was ...,android,0,"[Ca, n't, store, contacts, on, my, Android, ph..."
4,Dropbox on Samsung Galaxy - where is the Setti...,android,0,"[Dropbox, on, Samsung, Galaxy, -, where, is, t..."
...,...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D80...,android,0,"[How, to, securely, root, g2, phone, (, AT, &,..."
51366,Is Fennec F-Droid an official product by Mozil...,android,0,"[Is, Fennec, F-Droid, an, official, product, b..."
51367,Whats the difference between cell phone and da...,android,0,"[Whats, the, difference, between, cell, phone,..."
51368,“There are no android phones associated with t...,android,0,"[“, There, are, no, android, phones, associate..."


In [32]:
stop_words = set(stopwords.words('english'))

In [33]:
def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens
copy['merged_column_tokens'] = copy['merged_column_tokens'].apply(remove_stopwords)

In [34]:
copy

,merged_column,Label,LabelNum,merged_column_tokens
0,Drop\stop mobile data connection (non-wifi) by...,android,0,"[Drop\stop, mobile, data, connection, (, non-w..."
1,How to automatically crop text messages when S...,android,0,"[automatically, crop, text, messages, SMS, cha..."
2,Can't find text message that was to a groupWhe...,android,0,"[Ca, n't, find, text, message, groupWhen, John..."
3,Can't store contacts on my Android phoneI was ...,android,0,"[Ca, n't, store, contacts, Android, phoneI, go..."
4,Dropbox on Samsung Galaxy - where is the Setti...,android,0,"[Dropbox, Samsung, Galaxy, -, Settings, button..."
...,...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D80...,android,0,"[securely, root, g2, phone, (, &, LGE, LG-D800..."
51366,Is Fennec F-Droid an official product by Mozil...,android,0,"[Fennec, F-Droid, official, product, Mozilla, ..."
51367,Whats the difference between cell phone and da...,android,0,"[Whats, difference, cell, phone, data, radiati..."
51368,“There are no android phones associated with t...,android,0,"[“, android, phones, associated, account., ”, ..."


In [35]:
def remove_punctuation(tokens):
  punctuation_pattern = re.compile(r'[^\w\s]')
  filtered_tokens = [punctuation_pattern.sub('', token) for token in tokens]
  filtered_tokens = [token for token in filtered_tokens if token]
  return filtered_tokens

copy['merged_column_tokens'] = copy['merged_column_tokens'].apply(remove_punctuation)


In [36]:
copy

,merged_column,Label,LabelNum,merged_column_tokens
0,Drop\stop mobile data connection (non-wifi) by...,android,0,"[Dropstop, mobile, data, connection, nonwifi, ..."
1,How to automatically crop text messages when S...,android,0,"[automatically, crop, text, messages, SMS, cha..."
2,Can't find text message that was to a groupWhe...,android,0,"[Ca, nt, find, text, message, groupWhen, John,..."
3,Can't store contacts on my Android phoneI was ...,android,0,"[Ca, nt, store, contacts, Android, phoneI, goi..."
4,Dropbox on Samsung Galaxy - where is the Setti...,android,0,"[Dropbox, Samsung, Galaxy, Settings, button, S..."
...,...,...,...,...
51365,How to securely root g2 phone (AT&T LGE LG-D80...,android,0,"[securely, root, g2, phone, LGE, LGD800, ve, l..."
51366,Is Fennec F-Droid an official product by Mozil...,android,0,"[Fennec, FDroid, official, product, Mozilla, F..."
51367,Whats the difference between cell phone and da...,android,0,"[Whats, difference, cell, phone, data, radiati..."
51368,“There are no android phones associated with t...,android,0,"[android, phones, associated, account, recentl..."


In [37]:
from collections import Counter
all_tokens = [token for sublist in copy['merged_column_tokens'] for token in sublist]
word_freq = Counter(all_tokens)
print("Most common words and their frequencies:")
for word, freq in word_freq.most_common(10):
    print(f"{word}: {freq}")


Most common words and their frequencies:
phone: 43649
nt: 38173
app: 28560
Android: 26354
s: 22617
iPhone: 20658
device: 18890
apps: 15892
using: 15718
m: 15483


In [54]:
from collections import defaultdict

document_frequency = defaultdict(int)


for tokens in copy['merged_column_tokens']:
    unique_tokens = set(tokens)
    for token in unique_tokens:
        document_frequency[token] += 1


df_document_frequency = pd.DataFrame(document_frequency.items(), columns=['Token', 'Document Frequency'])

print(df_document_frequency)

                 Token  Document Frequency
0                limit                 378
1                  set                3968
2              version                4479
3              feature                1785
4               mobile                2256
...                ...                 ...
131970     dataWiFiLTE                   1
131971     radiationIs                   1
131972         eLocity                   1
131973  SettingsMobile                   1
131974    datacalltext                   1

[131975 rows x 2 columns]


In [53]:
document_frequency = defaultdict(int)


for tokens in copy['merged_column_tokens']:

    unique_tokens = set(tokens)
    for token in unique_tokens:
        document_frequency[token] += 1

total_documents = len(copy)
idf_values = {}
for token, frequency in document_frequency.items():
    idf_values[token] = math.log(total_documents / frequency)

idf_df = pd.DataFrame.from_dict(idf_values, orient='index', columns=['IDF'])
idf_df.index.name = 'Token'

print(idf_df)

                      IDF
Token                    
limit            4.911915
set              2.560792
version          2.439655
feature          3.359636
mobile           3.125461
...                   ...
dataWiFiLTE     10.846810
radiationIs     10.846810
eLocity         10.846810
SettingsMobile  10.846810
datacalltext    10.846810

[131975 rows x 1 columns]


In [55]:
idf_df_sorted = idf_df.sort_values(by='IDF')
print("Least common words in merged_column_tokens:")
print(idf_df_sorted.head())

Least common words in merged_column_tokens:
              IDF
Token            
nt       0.872398
phone    0.928778
Android  1.233541
s        1.284757
app      1.294015


In [42]:
counts = copy['LabelNum'].value_counts()
print("Count of Andriod:", counts.get(0, 0))
print("Count of IOS:", counts.get(1, 0))

Count of Andriod: 37153
Count of IOS: 14217
